In [1]:
from app.dotenv import base_dir, data_dir
video_fname = 'vtest.avi'
f"{data_dir}/{video_fname}"

'c:\\code\\identify_static_video_changes\\data/vtest.avi'

In [4]:
import numpy as np 
import cv2 
  
cap = cv2.VideoCapture(f"{data_dir}/{video_fname}") 
  
# initializing subtractor  
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()  
  
while(1): 
    ret, frame = cap.read()        
  
    # applying on each frame 
    fgmask = fgbg.apply(frame) 
      
  
    cv2.imshow('frame', fgmask) 
    k = cv2.waitKey(30) & 0xff
    if k == 27: 
        break
  
cap.release() 
cv2.destroyAllWindows() 

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


: 